# 🧠 BTC Price Prediction LSTM Model Training
This notebook trains an LSTM model on historical BTC-USD price data and saves it as `btc_model.h5`.

In [1]:
!pip install yfinance tensorflow scikit-learn


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


/Users/bibin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
2025-06-07 00:45:08.793133: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Download historical BTC/USD data
df = yf.download('BTC-USD', start='2020-01-01', end='2024-12-31')
df = df[['Close']].dropna()
data = df.values

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
# Create sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i - seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

SEQ_LEN = 60
X, y = create_sequences(scaled_data, SEQ_LEN)

# Reshape and split
X = X.reshape((X.shape[0], X.shape[1], 1))
train_size = int(len(X) * 0.8)
X_train, y_train = X[:train_size], y[:train_size]


In [5]:
# Build LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(SEQ_LEN, 1)),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')


/Users/bibin/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
# Train model
model.fit(X_train, y_train, epochs=5, batch_size=32)
model.save('btc_model.h5')
print("✅ Model trained and saved as btc_model.h5")


Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - loss: 0.0138
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - loss: 8.6580e-04
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 79ms/step - loss: 7.7331e-04
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 6.5765e-04
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - loss: 6.1168e-04


✅ Model trained and saved as btc_model.h5
